In [16]:
import fasttext as ft
from gensim.models import FastText, Word2Vec, KeyedVectors
import re
import string
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Masking, GRU
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint


In [17]:
#load file sample_for_chatbot_end.xlsx
def data(path):
    df = pd.read_excel(path, None)

    data = pd.DataFrame()
    for sheet_name in list(df.keys())[2:]:
        df = pd.read_excel(path, sheet_name, skiprows=4)
        data = pd.concat([data, df])
    data = data.fillna(method='ffill')
    data.columns = ['STT', 'Câu hỏi ban đầu', 'Câu hỏi', 'Trả lời']
    return data
    
data = data('sample_for_chatbot_end.xlsx')    

# split data into 2 part: test part with the first row of each intent and train part in other.
def split_train_test(data, label_table):
    
    data_train = data_test = pd.DataFrame(columns=data.columns)
    for i in label_table.unique():
        train_ = data[label_table==i].iloc[3:, :]
        test_ = data[label_table==i].iloc[:3, :]
        data_train = pd.concat([data_train, train_], axis=0)
        data_test = pd.concat([data_test, test_], axis=0)
        data_train = data_train.reset_index(drop=True)
        data_test = data_test.reset_index(drop=True)
    return data_train, data_test

data_train, data_test = split_train_test(data, data['Trả lời'])
#Data0 is the merge of train parts and test parts
data0 = pd.concat([data_train, data_test], axis=0)
data0 = data0.reset_index(drop=True)
data0.tail()

,STT,Câu hỏi ban đầu,Câu hỏi,Trả lời
385,19.0,Tôi muốn tìm lại lô hàng đã khai báo và thanh ...,có thể tìm lại lô hàng đã khai báo và thanh to...,"Sau khi đăng nhập, KH thực hiện như sau:\nB..."
386,19.0,Tôi muốn tìm lại lô hàng đã khai báo và thanh ...,làm thế nào để tìm lại lô hàng đã khai báo và ...,"Sau khi đăng nhập, KH thực hiện như sau:\nB..."
387,20.0,EDO là gì? DO và eDO khác nhau như thế nào?,EDO là gì?,EDO là lệnh giao hàng điện tử thay cho lê...
388,20.0,EDO là gì? DO và eDO khác nhau như thế nào?,sự khác biệt giữa eDo và DO,EDO là lệnh giao hàng điện tử thay cho lê...
389,20.0,EDO là gì? DO và eDO khác nhau như thế nào?,mối quan hệ giữa DO và eDO,EDO là lệnh giao hàng điện tử thay cho lê...


In [18]:
abbr_dict={
    "sai/mất":"sai",
    "cảng/icd":"cảng",
    "/icd":"cảng",
    "mst":"mã số thuế",
    "hủy":"bỏ"
}


In [19]:
#stop word in japanese: load into Japanese.txt
# def stop_word(path, is_jap):
#     if is_jap == True:
#         with open(path, 'r', encoding='utf-8') as file:
#             content = file.read()
#         stopword = content.split('\n')
#     else:
#         stopword = set(stopwords.words('english'))
#     return stopword   

# split sentences into word and remove punc, stopword
def split_text(str_):
    str_ = str_.lower()
    for key, values in abbr_dict.items():
        str_ = str_.replace(key, values)

    punc = list(string.punctuation)
#     stopword = stop_word('', False)
    word = word_tokenize(str_)
    word = [i for i in word if i not in punc]
    return word

# split_text("Công ty tôi là doanh nghiệp vận tải, chuyên làm dịch vụ giao nhận container cho các khách hàng tại cảng Tân Cảng Cát Lái. Khách hàng của tôi yêu cầu SNP xuất hóa đơn phí nâng/ hạ trực tiếp cho công ty của họ, vậy khi sử dụng E-port để đăng ký thủ tục và đóng phí nâng/ hạ, tôi phải thực hiện như thế nào để hóa đơn xuất ra trực tiếp cho khách hàng của tôi?")

In [20]:
#transfer sample question into list of word for train, test data
def transform_text(data):
    question_transform = data.apply(lambda x: split_text(x))
    return question_transform.values

question_transform = transform_text(data0['Câu hỏi'])
question_transform_train = transform_text(data_train['Câu hỏi'])
question_transform_test = transform_text(data_test['Câu hỏi'])


print(question_transform_test)

[list(['hỏi', 'thủ', 'tục', 'kéo', 'cont', 'về', 'kho', 'riêng'])
 list(['làm', 'thế', 'nào', 'để', 'kéo', 'cont', 'về', 'kho', 'riêng'])
 list(['thủ', 'tục', 'về', 'kéo', 'cont', 'về', 'kho', 'riêng', 'như', 'thế', 'nào'])
 list(['liên', 'hệ', 'bộ', 'phận', 'nào', 'để', 'đăng', 'ký', 'chuyển', 'thẳng', 'container', 'hàng', 'nhập', 'từ', 'tàu', 'sang', 'bãi', 'kiểm', 'hóa'])
 list(['muốn', 'chuyển', 'thẳng', 'container', 'hàng', 'nhập', 'từ', 'tàu', 'sang', 'bãi', 'kiểm', 'hóa', 'cần', 'liên', 'hệ', 'bộ', 'phận', 'nào'])
 list(['bộ', 'phận', 'hỗ', 'trợ', 'chuyển', 'thẳng', 'container', 'hàng', 'nhập', 'từ', 'tàu', 'sang', 'bãi', 'kiểm', 'hóa'])
 list(['mất', 'phiếu', 'xuất', 'nhập', 'bãi'])
 list(['xin', 'cấp', 'lại', 'phiếu', 'xuất', 'nhập', 'bãi'])
 list(['hỏi', 'thủ', 'tục', 'xin', 'cấp', 'lại', 'phiếu', 'xuất', 'nhập', 'bãi'])
 list(['hỏi', 'thủ', 'tục', 'đón', 'thẳng', 'container', 'hàng', 'từ', 'tàu', 'về', 'kho'])
 list(['hỏi', 'quy', 'trình', 'đón', 'thằng', 'container', 'hàng'

In [21]:
def model(path, limit):
    model = KeyedVectors.load_word2vec_format(path, limit=limit)
    return model

fasttext = model('./pre_train/cc.vi.300.vec', 300000)

In [22]:
a = fasttext['bỏ']
print(a)

[-1.0000e-02  2.2900e-02 -1.6050e-01 -1.4900e-02  1.7560e-01 -2.0970e-01
  1.9090e-01  1.7200e-02 -2.0100e-01 -3.1310e-01 -7.3200e-02 -5.5100e-02
  4.2150e-01  1.0960e-01  1.7180e-01 -1.4710e-01  1.1590e-01  5.9400e-02
 -5.5000e-03 -1.4700e-02 -1.7890e-01 -6.3100e-02  1.0530e-01  1.6200e-01
  4.2000e-03 -2.6530e-01 -3.9040e-01  2.2300e-02  3.1750e-01  1.8900e-01
 -1.2800e-01 -1.6180e-01  1.5050e-01  1.8000e-01  5.7600e-02  4.3000e-03
  1.3600e-01  3.4480e-01 -9.4000e-03  1.9600e-01  9.6700e-02 -8.2500e-02
  2.3160e-01 -7.6000e-02  1.9290e-01  1.4900e-01 -2.6630e-01  6.5900e-02
  2.4160e-01 -1.6970e-01 -5.2100e-02 -2.8860e-01  1.4450e-01 -3.7500e-02
 -8.3000e-03 -2.5000e-02 -5.6600e-02  4.3900e-02 -2.2660e-01 -2.0690e-01
  4.1100e-02 -2.4490e-01  2.2620e-01  3.1800e-02 -1.0170e-01  1.4610e-01
  2.5770e-01 -6.3000e-03 -1.5600e-01  1.0170e-01  2.1100e-01 -2.1680e-01
  3.7100e-02  9.4200e-02 -2.0950e-01 -5.0600e-02  2.1800e-01  1.2600e-01
  7.5000e-02 -9.6300e-02 -7.4200e-02 -2.5050e-01  8

In [23]:
#list of vocab in a columns dataframe

def listword(df):
    list_word = [i for j in df for i in j]
    return list_word
list_word = listword(question_transform)

#list vocab from sample question
def vocab(df):
    list_word_set = list(set(df))
    return list_word_set

list_word_set = vocab(list_word)
print(list_word_set)

['dụng', 'dàng', 'sẽ', 'nhập', 'quên', 'đầu', 'lại', 'online', 'cá', 'trực', 'ưu', 'khoản', 'giải', 'đổi', 'lý', 'thái', 'đơn', 'dẫn', 'thế', 'lấy', 'thực', 'sách', 'gặp', 'tử', 'lũy', 'tục', 'mật', 'khách', 'dù', 'sau', 't', 'cương', 'giữa', 'loại', 'thông', 'quan', 'cơ', 'bỏ', 'liệu', 'tại', 'tư', 'gia', 'đúng', 'xuất', 'tiếp', 'mạng', 'tin', 'phận', 'xem', 'muốn', 'trang', 'in', 'quyết', 'hỗ', 'xác', 'hiệu', 'khai', 'bị', 'cầu', 'nhiệm', 'kho', 'túc', 'nâng', 'phiếu', 'thành', 'tục', 'lấy', 'nội', 'hóa', 'đãi', 'gọi', 'gửi', 'kháo', 'tín', 'mà', 'tôi', 'nhưng', 'tần', 'lâu', 'tiền', 'đóng', 'hiện', 'vẫn', 'chi', 'lỗi', 'thì', 'lợi', 'của', 'các', 'càng', 'trực', 'đảm', 'khiếu', 'cả', 'lệ', 'mật', 'quốc', 'cách', 'phát', 'rút', 'và', 'giống', 'sửa', 'điện', 'tiện', 'thẳng', 'đến', 'bạc', 'liên', 'giảm', 'hệ', 'cảng', 'danh', 'soi', 'tiền', 'hướng', 'giản', 'từ', 'công', 'xong', 'eport', 'toán', 'ai', 'thức', 'đãn', 'lượng', 'xử', 'chiết', 'làm', 'ứng',

In [24]:
vocab_ = list(fasttext.vocab) #list vocab in gensim --> not enough word from sample question list
print(len(vocab_))
not_enough = [i for i in list_word_set if i not in vocab_]

print('list vocab in sample question:', list_word_set)
print('-'*50)
print('The word is miss:', not_enough)


300000
list vocab in sample question: ['dụng', 'dàng', 'sẽ', 'nhập', 'quên', 'đầu', 'lại', 'online', 'cá', 'trực', 'ưu', 'khoản', 'giải', 'đổi', 'lý', 'thái', 'đơn', 'dẫn', 'thế', 'lấy', 'thực', 'sách', 'gặp', 'tử', 'lũy', 'tục', 'mật', 'khách', 'dù', 'sau', 't', 'cương', 'giữa', 'loại', 'thông', 'quan', 'cơ', 'bỏ', 'liệu', 'tại', 'tư', 'gia', 'đúng', 'xuất', 'tiếp', 'mạng', 'tin', 'phận', 'xem', 'muốn', 'trang', 'in', 'quyết', 'hỗ', 'xác', 'hiệu', 'khai', 'bị', 'cầu', 'nhiệm', 'kho', 'túc', 'nâng', 'phiếu', 'thành', 'tục', 'lấy', 'nội', 'hóa', 'đãi', 'gọi', 'gửi', 'kháo', 'tín', 'mà', 'tôi', 'nhưng', 'tần', 'lâu', 'tiền', 'đóng', 'hiện', 'vẫn', 'chi', 'lỗi', 'thì', 'lợi', 'của', 'các', 'càng', 'trực', 'đảm', 'khiếu', 'cả', 'lệ', 'mật', 'quốc', 'cách', 'phát', 'rút', 'và', 'giống', 'sửa', 'điện', 'tiện', 'thẳng', 'đến', 'bạc', 'liên', 'giảm', 'hệ', 'cảng', 'danh', 'soi', 'tiền', 'hướng', 'giản', 'từ', 'công', 'xong', 'eport', 'toán', 'ai', 'thức', 'đãn', 

In [26]:
#Transfer label from column intent into one hot encoding
def one_hot_encodeing(feature):

    onehot = np.zeros((len(feature), feature.nunique()))
    feature_list = list(feature.unique())
    for intent in range(len(feature)):
        index = feature_list.index(feature[intent])
        onehot[intent, index] = 1
    return onehot, feature_list

train_onehot, train_feature = one_hot_encodeing(data_train['Trả lời'])
test_onehot, test_feature = one_hot_encodeing(data_test['Trả lời'])
onehot_, feature_ = one_hot_encodeing(data0['Trả lời'])
print(train_onehot.shape)
print(test_onehot.shape)
print(onehot_.shape)

(273, 39)
(117, 39)
(390, 39)


In [28]:
#Creat a matrix from sample question with the dimention is row x max_lenght x 100
def word_embed(df, model, max_length):
    embed_all = []
    for word_split in df:

        if len(word_split) > 0:
            embed_token = []
            for word in word_split:
                if word in vocab_:
 
                    vec = model[word]
                    embed_token.append(vec)

            if len(embed_token) != 0:
                if len(embed_token) >= max_length: #if len(embed_token) > max_length then remove in larger
                    embed_token = embed_token[:max_length]
                else: #if len(embed_token) < max_length then add 0 for enough
                    add_vect = list(np.zeros((max_length - len(embed_token), 300)))
                    embed_token = np.concatenate((embed_token, add_vect))

            else:
                print(False)

        else:
            print(False)
        embed_all.append(embed_token)
    return np.asarray(embed_all)
#Train, test data to vector
all_ = word_embed(question_transform, fasttext, 20)
train = word_embed(question_transform_train, fasttext, 20)
test = word_embed(question_transform_test, fasttext, 20)

print(train.shape)
print(test.shape)

(273, 20, 300)
(117, 20, 300)


In [49]:
#creat LSTM model with input_shape = max_lengh x 100
def LSTM_(X, y, input_dim, epoch, batch_size, output_dim):
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=(input_dim)))
    model.add(Dropout(0.2))
    model.add(GRU(units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(units=output_dim, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='Adam', 
                 metrics=['accuracy'])
    if not os.path.exists('keras_model'):
        os.makedirs('keras_model')
    save_weight = os.path.join('keras_model', 'LSTM_{loss:.4f}.hdf5')
    save_best = ModelCheckpoint(save_weight, monitor='loss', save_best_only=True, save_weights_only=False, mode='min', period=1)
    
    model.summary()
    
    history = model.fit(X, y, epochs=epoch, batch_size=batch_size, callbacks=[save_best])
    
    return model



Creat Model check points for model save best loss in each epoch, if loss new smaller than all file saved, it will save a new file

In [50]:
# train with epochs = 1000 and batch_size = 32
input_dim = (train.shape[1],train.shape[2])
model = LSTM_(train, train_onehot, input_dim, 1000, 16, onehot_.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20, 128)           219648    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
gru_4 (GRU)                  (None, 20, 64)            37056     
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 64)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 39)                1287      
Total para

273/273 [==============================] - 1s 3ms/step - loss: 0.1240 - acc: 1.0000
Epoch 72/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.1155 - acc: 1.0000
Epoch 73/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.1176 - acc: 1.0000
Epoch 74/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.1132 - acc: 0.9963
Epoch 75/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.1140 - acc: 1.0000
Epoch 76/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.1079 - acc: 1.0000
Epoch 77/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.1098 - acc: 1.0000
Epoch 78/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0999 - acc: 1.0000
Epoch 79/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.1006 - acc: 1.0000
Epoch 80/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.1020 - acc: 1.0000
Epoch 81/1000
273/273 [===========

273/273 [==============================] - 1s 3ms/step - loss: 0.0380 - acc: 1.0000
Epoch 154/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0364 - acc: 1.0000
Epoch 155/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0356 - acc: 1.0000
Epoch 156/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0376 - acc: 1.0000
Epoch 157/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0350 - acc: 1.0000
Epoch 158/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0333 - acc: 1.0000
Epoch 159/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0337 - acc: 1.0000
Epoch 160/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0303 - acc: 1.0000
Epoch 161/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0317 - acc: 1.0000
Epoch 162/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0281 - acc: 1.0000
Epoch 163/1000
273/273 [=

273/273 [==============================] - 1s 3ms/step - loss: 0.0119 - acc: 1.0000
Epoch 236/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0125 - acc: 1.0000
Epoch 237/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0120 - acc: 1.0000
Epoch 238/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0131 - acc: 1.0000
Epoch 239/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0139 - acc: 1.0000
Epoch 240/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0130 - acc: 1.0000
Epoch 241/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0118 - acc: 1.0000
Epoch 242/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0161 - acc: 1.0000
Epoch 243/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0113 - acc: 1.0000
Epoch 244/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0133 - acc: 1.0000
Epoch 245/1000
273/273 [=

273/273 [==============================] - 1s 3ms/step - loss: 0.0062 - acc: 1.0000
Epoch 318/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0057 - acc: 1.0000
Epoch 319/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0091 - acc: 1.0000
Epoch 320/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0068 - acc: 1.0000
Epoch 321/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0075 - acc: 1.0000
Epoch 322/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0059 - acc: 1.0000
Epoch 323/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0059 - acc: 1.0000
Epoch 324/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0069 - acc: 1.0000
Epoch 325/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0074 - acc: 1.0000
Epoch 326/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0058 - acc: 1.0000
Epoch 327/1000
273/273 [=

...

273/273 [==============================] - 1s 3ms/step - loss: 0.0026 - acc: 1.0000
Epoch 646/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0024 - acc: 1.0000
Epoch 647/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0029 - acc: 1.0000
Epoch 648/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0030 - acc: 1.0000
Epoch 649/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0022 - acc: 1.0000
Epoch 650/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0023 - acc: 1.0000
Epoch 651/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0033 - acc: 1.0000
Epoch 652/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0035 - acc: 1.0000
Epoch 653/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0021 - acc: 1.0000
Epoch 654/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0026 - acc: 1.0000
Epoch 655/1000
273/273 [=

273/273 [==============================] - 1s 3ms/step - loss: 0.0061 - acc: 1.0000
Epoch 728/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0062 - acc: 1.0000
Epoch 729/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0092 - acc: 1.0000
Epoch 730/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0052 - acc: 1.0000
Epoch 731/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0037 - acc: 1.0000
Epoch 732/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0045 - acc: 1.0000
Epoch 733/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0060 - acc: 1.0000
Epoch 734/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0058 - acc: 1.0000
Epoch 735/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0045 - acc: 1.0000
Epoch 736/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0053 - acc: 1.0000
Epoch 737/1000
273/273 [=

273/273 [==============================] - 1s 3ms/step - loss: 0.0583 - acc: 0.9890
Epoch 810/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0462 - acc: 0.9890
Epoch 811/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0409 - acc: 0.9890
Epoch 812/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0431 - acc: 0.9927
Epoch 813/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0356 - acc: 0.9963
Epoch 814/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0366 - acc: 0.9927
Epoch 815/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0228 - acc: 1.0000
Epoch 816/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0221 - acc: 0.9963
Epoch 817/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0190 - acc: 1.0000
Epoch 818/1000
273/273 [==============================] - 1s 4ms/step - loss: 0.0189 - acc: 1.0000
Epoch 819/1000
273/273 [=

273/273 [==============================] - 1s 3ms/step - loss: 0.0039 - acc: 1.0000
Epoch 892/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0030 - acc: 1.0000
Epoch 893/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0034 - acc: 1.0000
Epoch 894/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0049 - acc: 1.0000
Epoch 895/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0069 - acc: 1.0000
Epoch 896/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0037 - acc: 1.0000
Epoch 897/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0034 - acc: 1.0000
Epoch 898/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0046 - acc: 1.0000
Epoch 899/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0036 - acc: 1.0000
Epoch 900/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0043 - acc: 1.0000
Epoch 901/1000
273/273 [=

273/273 [==============================] - 1s 3ms/step - loss: 0.0020 - acc: 1.0000
Epoch 974/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0020 - acc: 1.0000
Epoch 975/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0015 - acc: 1.0000
Epoch 976/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0015 - acc: 1.0000
Epoch 977/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0020 - acc: 1.0000
Epoch 978/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0024 - acc: 1.0000
Epoch 979/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0015 - acc: 1.0000
Epoch 980/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0026 - acc: 1.0000
Epoch 981/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0015 - acc: 1.0000
Epoch 982/1000
273/273 [==============================] - 1s 3ms/step - loss: 0.0040 - acc: 1.0000
Epoch 983/1000
273/273 [=

In [51]:
#sort the model saved and load the best model is the first list file
def load_(path):
    list_ = os.listdir(path)
    list_.sort()
    file_name = os.path.join(path, list_[0])
    print(file_name)
    model_predict = load_model(file_name, '')
    return model_predict

model_predict = load_('keras_model')

keras_model\LSTM_0.0011.hdf5


In [52]:
#evaluate model
def evaluate_model(X, y, model):
    loss = model.evaluate(X, y, verbose=0)
    return loss

evaluate_model(test, test_onehot, model_predict)

[1.3532446449638431, 0.8461538481916118]

In [53]:
def predict(sent, model, model_predict, max_length, threshold):
    if sent == '':
        result = 'not empty'
    else:
        split_ = split_text(sent)
        vect = word_embed([split_], model, max_length)
        predict = model_predict.predict(vect)
        max_prob = np.max(predict)
        if max_prob >= threshold: #add threshold to predict sent
            result = train_feature[np.argmax(predict[0])]
        else:
            result = 'No meaning'
    return result

display again result of test data

In [54]:
for index in range(data_test.shape[0]):
    sent = data_test.loc[index, 'Câu hỏi']
    actual = data_test.loc[index, 'Trả lời']
    pre = predict(sent, fasttext, model_predict, 20, 0)
    print('sentences:', sent)
    print('predict:', pre)
    print('actual:', actual)
    if actual == pre:
        print(True)
    else:
        print(False)
    print('-'*100)
    
print(predict('Cách để lấy hóa đơn', fasttext, model_predict, 20, 0.5))

sentences: hỏi thủ tục kéo cont về kho riêng
predict: 1. Khách hàng làm công văn nêu rõ lý do có dấu và chữ ký của Chủ doanh nghiệp trình Hãng tàu, Hải quan và Cảng.
2. Khách hàng xin xác nhận đồng ý của Hãng tàu (chủ khai thác container).
3. Khách hàng xin xác nhận chưa vào sổ tàu hoặc hủy đăng ký sổ tàu tại bộ phận Đăng ký tàu xuất.
4. Khách hàng xin xác nhận đồng ý cho lấy hàng ra khỏi Cảng của cơ quan Hải quan:
- Công chức Hải quan Giám sát bãi
- Chỉ huy đội Hải quan Giám sát
- Lãnh đạo Chi cục Hải quan cửa khẩu Cảng Sài gòn Khu vực I - Cát lái
5. Tại TBSX, nhân viên TBKHĐH liên hệ Trực ban Terminal/ Khu hàng quản lý container để xác nhận vị trí và số lượng container đảo chuyển thực tế cho Khách hàng.
- Đối với số lượng lấy ra khỏi cảng ≤ 03 container, TBKHĐH xem xét giải  quyết.
- Đối với số lượng lấy ra khỏi cảng 04 đến 05 container TBKHĐH trình BGD TTDDC xem xét giải quyết.
- Đối với số lượng lấy ra khỏi cảng trên 05 container  TBKHĐH hướng dẫn Khách hàng mang bộ chứng từ trình 

sentences: Hỏi về các trường hợp đóng phí lũy tiến khi đóng tiền nâng hạ tại cảng
predict: Phí lũy tiến là phí chỉ áp dụng đối với cont hàng nhập từ ngày thứ 7 sau khi cont đã được xếp vào bãi. Đối với cont hàng nhập thông thường từ ngày thứ 7 trở đi, khách hàng phải đóng phí lũy tiến khi đến lấy cont tại Cảng. Phí lũy tiến được tính chia làm 02 mức :
• Mức 1 : từ ngày thứ 7 đế ngày thứ 15.
• Mức 2 : từ ngày thứ 16 trở đi.

actual: Phí lũy tiến là phí chỉ áp dụng đối với cont hàng nhập từ ngày thứ 7 sau khi cont đã được xếp vào bãi. Đối với cont hàng nhập thông thường từ ngày thứ 7 trở đi, khách hàng phải đóng phí lũy tiến khi đến lấy cont tại Cảng. Phí lũy tiến được tính chia làm 02 mức :
• Mức 1 : từ ngày thứ 7 đế ngày thứ 15.
• Mức 2 : từ ngày thứ 16 trở đi.

True
----------------------------------------------------------------------------------------------------
sentences: Các trường hợp đóng phí lũy tiếng khi đóng tiền nâng hạ tại cảng là gì
predict: Phí lũy tiến là phí chỉ áp dụn

sentences: Các thủ tục cần thiết để rút hàng từ container này sang container khác
predict: KH vui lòng đăng ký qua địa chỉ mail: khuhang.kbcl@saigonnewport.com.vn ít nhất 4h trước khi tàu cập Cảng.
Nội dung đăng ký : tên, địa chỉ, số điện thoại của công ty hoặc cá nhân yêu cầu; số container; kích cỡ; tên tàu; chuyến tàu; ngày cập; ngày dự kiến nhận hàng; cảng đích, ghi chú (nếu có)
Điện thoại: 028 37423917/ Ext: 2658   

actual: * Khách hàng thực hiện những công việc sau:
1. Khách hàng làm công văn nêu rõ lý do có dấu và chữ ký của Chủ doanh nghiệp trình Hãng tàu, Hải quan và Cảng.
2. Khách hàng xin xác nhận đồng ý của Hãng tàu (chủ khai thác container).
3. Khách hàng xin xác nhận đồng ý cho rút ruột sang container mới của cơ quan Hải quan: 
- Công chức Hải quan Giám sát.
- Chỉ huy đội Hải quan Giám sát.
- Lãnh đạo Chi cục Hải quan cửa khẩu Cảng Sài gòn Khu vực I - Cát lái.
4. Khách hàng xin xác nhận chưa vào sổ tàu hoặc hủy đăng ký sổ tàu tại bộ phận Đăng ký tàu xuất.
5. Tại Khu bãi h

actual: Các video hướng dẫn cách khai báo và thanh toán E-port:
• Hàng nhập giao thẳng có sử dụng eDO
https://www.youtube.com/watch?v=jw4x58keGRc&feature=youtu.be
• Hàng nhập giao thẳng không sử dụng eDO 
https://www.youtube.com/watch?v=hny3o6-FL8I&feature=youtu.be
• Cấp rỗng thường 
https://www.youtube.com/watch?v=-lQ-m_cyI4Q&feature=youtu.be
• Cấp rỗng chỉ định số 
https://www.youtube.com/watch?v=pFcR0J-nG7I&feature=youtu.be
• Thực hiện tách lô từ Master Bill 
https://www.youtube.com/watch?v=umIgnf-cq4E&feature=youtu.be
• Hàng xuất hạ bãi chờ xuất 
https://www.youtube.com/watch?v=keIzEPE9yJ8&feature=youtu.be
• Hàng nguy hiểm hạ bãi chờ xuất
https://www.youtube.com/watch?v=7yywGtq9vjc&feature=youtu.be
• Hàng OOG và In-gauge hạ bãi chờ xuất 
https://www.youtube.com/watch?v=Bny_C6Hz1aM&feature=youtu.be
• Hàng lạnh cài nhiệt độ & không cài nhiệt độ hạ bãi chờ xuất 
https://www.youtube.com/watch?v=UasApmVkt2w&feature=youtu.be
• Trả rỗng từ cont hàng nhập trước đó dùng e-DO 
https://www.yo

sentences: E-port báo thể không hợp lệ
predict: KH vui lòng gửi email thông tin MST cần xuất hóa đơn đến địa chỉ tct.eport@saigonnewport.com.vn VÀ eport.phct@saigonnewport.com.vn để cập nhật dữ liệu trên hệ thống của Cảng. 
Trường hợp đã gửi email mà Cảng chưa phản hồi (ban đêm hoặc ngày nghỉ, lễ), đề nghị KH gửi mail vào địa chỉ ttcskh@saigonnewport.com.vn để được hỗ trợ tiếp. 

actual: Như quý khách hàng đã biết, nhằm khuyến khích quý khách hàng thực hiện việc đăng ký thủ tục và thanh toán trực tuyến trên E-port, SNP có chính sách hỗ trợ chi phí cho quý khách hàng bao gồm:
1. Hỗ trợ quý khách hàng phí giao dịch qua NAPAS, phí giao dịch qua ngân hàng (1.600đ/ giao dịch qua NAPAS và 1% giá trị giao dịch đối với thanh toán bằng thẻ ATM nội địa; hoặc phí ngân hàng đối với giao dịch qua E-banking).
2. Chiết khấu 2% trên đơn giá dịch vụ tại cảng (chiết khấu trực tiếp trên hóa đơn hoặc chiết khấu vào thẻ của đơn vị thanh toán).
Như vậy chi phí mà SNP thực chi bao gồm 2 phần: chi phí giao 

sentences: nhận cont từ cảng và các việc phải làm
predict: Đối với KH yêu cầu chuyển máy soi: Eport tạm thời chưa hỗ trợ khai báo. KH có nhu cầu chuyển soi sau khi khai báo eport giao thẳng, KH vào quầy PHCT đăng ký và đóng tiền chuyển soi sau đó sang quầy kiểm hóa để đk chuyển soi.
Đối với KH kiểm hóa: tác nghiệp kiểm hóa không yêu cầu đóng tiền trước, do đó KH có thể đky kiểm hóa trước. Khi đó bộ phận kiẻm hóa sẽ ghi nhận lên hệ thống tác nghiệp phát sinh. Khi KH khai báo lấy hàng nhập qua eport, hóa đơn giao thẳng sẽ bao gồm phí kiểm hóa.


actual: Đối với KH yêu cầu chuyển máy soi: Eport tạm thời chưa hỗ trợ khai báo. KH có nhu cầu chuyển soi sau khi khai báo eport giao thẳng, KH vào quầy PHCT đăng ký và đóng tiền chuyển soi sau đó sang quầy kiểm hóa để đk chuyển soi.
Đối với KH kiểm hóa: tác nghiệp kiểm hóa không yêu cầu đóng tiền trước, do đó KH có thể đky kiểm hóa trước